# High-level Arm test

This code will test the arm over bluetooth, and also test kinematics on the arm.

In [ ]:
%load_ext autoreload
%autoreload 2
%aimport dynamixel
%aimport kinematics

In [ ]:
from arm import Arm, JointPathWaypoint, PosePathWaypoint
import time
import numpy as np
from util import SE3

In [ ]:
# PORT, BAUD, WRITE_PACKET_DELAY = '/dev/tty.borgla_test', 9600, 0.15
# PORT, BAUD, WRITE_PACKET_DELAY = '/dev/tty.usbmodem1101', 9600, 0.01
PORT, BAUD, WRITE_PACKET_DELAY, READ_ALL_TIMEOUT = '/dev/tty.usbmodem103568505', 1000000, 0.002, 0.002

Robot = lambda **kwargs: Arm(
    PORT, BAUD, write_packet_delay=WRITE_PACKET_DELAY, read_all_timeout=READ_ALL_TIMEOUT, **kwargs)

In [ ]:
# Test connection to all servos by pinging
# We expect print 6 lines, all with error=0.  See https://emanual.robotis.com/docs/en/dxl/protocol1/#error
with Robot() as robot:
    robot.ping(0)
    robot.ping(1)
    robot.ping(2)
    robot.ping(3)
    robot.ping(4)
    robot.ping(5)
    for msg in robot.read_all_msgs():
        print(msg)

In [ ]:
# Allow hand-positioning of arm and print joint angles
with Robot() as robot:
    robot.disable_all()
    while input() != 'q':
        print(robot.joint_angles_deg())

In [ ]:
# Check FK by disabling motors and printing tip position as the robot is moved by hand
with Robot() as robot:
    robot.disable_all()
    while True:
        try:
            print(robot, end='\r')
        except KeyboardInterrupt:
            time.sleep(0.5)  # Give some time for unpaired responses to come back
            robot.clear_buffer()
            break
    print()
    print('Pose:')
    print(robot.cur_pose())

In [ ]:
# Set arm to 0 position
with Robot() as robot:
    robot.enable_all()
    robot.do_move_home(verbosity=2)

In [ ]:
with Robot(print_warnings=False) as robot:
    robot.enable_all()
    robot.do_dip(verbosity=2)

In [ ]:
with Robot(print_warnings=False) as robot:
    robot.enable_all()
    robot.do_prep_paint(ease_dist=.1, elbow_mode='neg')

In [ ]:
with Robot(print_warnings=False) as robot:
    robot.enable_all()
    robot.do_start_paint()
    input()
    robot.do_prep_paint()

In [ ]:
robot.do_move_home()

In [ ]:
# Try tilting the brush
Ts = [(SE3.P(Arm.CANVAS_CENTER + [0, 0, 0.06]) @ SE3.RotX(-np.pi / 2 + angle))
      for angle in np.linspace(0, .5, 5)]
path = [PosePathWaypoint(T=T, tol=5, timeout=None, pause=0) for T in Ts]

with Robot(print_warnings=False) as robot:
    robot.execute_pose_path(path)